In [67]:

import pandas as pd

In [68]:
df_train = pd.read_csv('00_train_10_V3.csv')
df_test = pd.read_csv('00_test_10_V3.csv')
df_validation = pd.read_csv('00_validation_10_V3.csv')

FileNotFoundError: [Errno 2] No such file or directory: '00_train_10_V3.csv'

In [2]:
import numpy as np
import pandas as pd
import ast
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense

# --- Chargement et prétraitement des données ---

def parse_cell(cell):
    return np.array(ast.literal_eval(cell), dtype=np.int32)

def df_to_tensor(df, input_cols=200):
    X = np.stack([
        np.stack([parse_cell(df.iloc[i, j]) for j in range(input_cols)], axis=0)
        for i in range(len(df))
    ])
    y = np.stack([parse_cell(df.iloc[i, input_cols]) for i in range(len(df))])
    return X, y

# Charge les CSV (adapter les chemins si besoin)
df_train = pd.read_csv('00_train_10_V6.csv')
df_val = pd.read_csv('00_validation_10_V6.csv')
df_test = pd.read_csv('00_test_10_V6.csv')

X_train, y_train = df_to_tensor(df_train)
X_val, y_val = df_to_tensor(df_val)
X_test, y_test = df_to_tensor(df_test)

# --- Définition du modèle ---

model = Sequential([
    Input(shape=(200, 10)),
    LSTM(256),
    Dense(128, activation='relu'),
    Dense(10)
])

model.compile(optimizer='adam', loss='mse')

# --- Entraînement ---
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=32)


ValueError: malformed node or string: 46

In [ ]:
def generate_next_chord(model, X_seed):
    """
    Génère un accord (liste de 10 notes) à partir d'une séquence de 200 accords.
    
    Paramètres :
    - model : le modèle LSTM entraîné
    - X_seed : np.ndarray de shape (200, 10), représentant une séquence d'accords
    
    Retour :
    - next_chord : np.ndarray de shape (10,), représentant l'accord prédit
    """
    if X_seed.shape != (200, 10):
        raise ValueError("X_seed doit être de shape (200, 10)")
    
    # Ajouter la dimension batch
    X_input = np.expand_dims(X_seed, axis=0)  # (1, 200, 10)
    
    # Prédiction
    y_pred = model.predict(X_input, verbose=0)
    
    # Résultat : vecteur de 10 notes
    return y_pred[0]


In [ ]:
# Exemple avec le premier échantillon du test set
X_seed = X_test[0]  # shape (200, 10)
predicted_chord = generate_next_chord(model, X_seed)

print("Accord généré :", predicted_chord)


Accord généré : [ 8.7720482e-04  5.8262728e-02 -9.6138626e-01 -8.7486470e-01
 -9.3807757e-01 -1.0373958e+00 -9.7606647e-01 -9.7141653e-01
 -9.8649287e-01 -9.9443847e-01]


In [ ]:
X_test[0]

array([[ 0, -1, -1, ..., -1, -1, -1],
       [ 0, -1, -1, ..., -1, -1, -1],
       [ 0, -1, -1, ..., -1, -1, -1],
       ...,
       [ 0, -1, -1, ..., -1, -1, -1],
       [ 0,  0,  0, ..., -1, -1, -1],
       [ 0,  0,  0, ..., -1, -1, -1]])

In [ ]:
def generate_chord_sequence(model, seed_sequence, num_chords=50):
    """
    Génère une séquence d'accords à partir d'une séquence initiale (seed).
    
    Paramètres :
    - model : le modèle LSTM entraîné
    - seed_sequence : np.ndarray de shape (200, 10), séquence de départ
    - num_chords : int, nombre d'accords à générer
    
    Retour :
    - generated : np.ndarray de shape (num_chords, 10)
    """
    generated = []
    current_sequence = seed_sequence.copy()  # (200, 10)

    for _ in range(num_chords):
        next_chord = generate_next_chord(model, current_sequence)  # (10,)
        generated.append(next_chord)
        # Mettre à jour la séquence courante : supprimer le 1er accord et ajouter le nouveau
        current_sequence = np.vstack([current_sequence[1:], next_chord])

    return np.array(generated)  # (num_chords, 10)

# Exemple d'utilisation :
seed = X_test[0]  # On prend la première séquence du jeu de test comme seed
generated_sequence = generate_chord_sequence(model, seed, num_chords=50)

# Afficher les accords générés
for i, chord in enumerate(generated_sequence):
    print(f"Accord {i+1:02d} :", np.round(chord).astype(int))


Accord 01 : [ 0  0 -1 -1 -1 -1 -1 -1 -1 -1]
Accord 02 : [ 0  0  0  0 -1 -1 -1 -1 -1 -1]
Accord 03 : [ 0  0 -1 -1 -1 -1 -1 -1 -1 -1]
Accord 04 : [ 0  0  0 -1 -1 -1 -1 -1 -1 -1]
Accord 05 : [ 0  0  0 -1 -1 -1 -1 -1 -1 -1]
Accord 06 : [ 0  0 -1 -1 -1 -1 -1 -1 -1 -1]
Accord 07 : [ 0  0  0 -1 -1 -1 -1 -1 -1 -1]
Accord 08 : [ 0  0  0 -1 -1 -1 -1 -1 -1 -1]
Accord 09 : [ 0  0  0 -1 -1 -1 -1 -1 -1 -1]
Accord 10 : [ 0  0  0 -1 -1 -1 -1 -1 -1 -1]
Accord 11 : [ 0  0 -1 -1 -1 -1 -1 -1 -1 -1]
Accord 12 : [ 0  0 -1  0 -1 -1 -1 -1 -1 -1]
Accord 13 : [ 0  0 -1 -1 -1 -1 -1 -1 -1 -1]
Accord 14 : [ 0  0  0 -1 -1 -1 -1 -1 -1 -1]
Accord 15 : [ 0  0  0 -1 -1 -1 -1 -1 -1 -1]
Accord 16 : [ 0  0 -1 -1 -1 -1 -1 -1 -1 -1]
Accord 17 : [ 0  0  0 -1 -1 -1 -1 -1 -1 -1]
Accord 18 : [ 0  0 -1 -1 -1 -1 -1 -1 -1 -1]
Accord 19 : [ 0  0 -1 -1 -1 -1 -1 -1 -1 -1]
Accord 20 : [ 0  0 -1 -1 -1 -1 -1 -1 -1 -1]
Accord 21 : [ 0  0 -1 -1 -1 -1 -1 -1 -1 -1]
Accord 22 : [ 0  0  0 -1 -1 -1 -1 -1 -1 -1]
Accord 23 : [ 0  0 -1 -1 -1 -1 -

In [ ]:
generated_sequence

array([[ 8.77204817e-04,  5.82627282e-02, -9.61386263e-01,
        -8.74864697e-01, -9.38077569e-01, -1.03739583e+00,
        -9.76066470e-01, -9.71416533e-01, -9.86492872e-01,
        -9.94438469e-01],
       [ 2.08601356e-04, -1.09899908e-01,  2.68512398e-01,
        -3.88254300e-02, -8.23088169e-01, -9.51898396e-01,
        -9.04173195e-01, -9.79840994e-01, -9.73519683e-01,
        -9.73327637e-01],
       [ 4.28332947e-04, -1.26963347e-01, -1.03400064e+00,
        -1.16579640e+00, -9.23973501e-01, -9.66409862e-01,
        -1.05821180e+00, -1.01904786e+00, -1.01070964e+00,
        -1.01565278e+00],
       [ 4.15610615e-04,  1.55655518e-01, -2.39999935e-01,
        -8.42073381e-01, -1.02608061e+00, -8.89826298e-01,
        -9.24998164e-01, -9.87385452e-01, -9.85626817e-01,
        -9.85259652e-01],
       [ 3.46137211e-04, -1.00979500e-01, -2.82589525e-01,
        -5.94959617e-01, -8.97555292e-01, -1.00147986e+00,
        -9.59410071e-01, -9.68228877e-01, -9.83690023e-01,
        -9.

In [ ]:
partition = []
for i in generated_sequence:
    ac = []
    for j in i:
        n = j*127
        if n>30:
            ac.append(round(n))
    partition.append(ac)

In [ ]:
notes_data = partition

In [ ]:
from music21 import stream, chord, midi

# Création du flux musical
s = stream.Stream()

# Ajout des accords/notes dans la partition
for note_group in notes_data:
    if len(note_group) == 1:
        n = chord.Chord(note_group)  # aussi possible d'utiliser note.Note
    else:
        n = chord.Chord(note_group)
    n.quarterLength = 1  # durée de chaque élément
    s.append(n)

# Export en fichier MIDI
mf = midi.translate.music21ObjectToMidiFile(s)
mf.open("test4.mid", 'wb')
mf.write()
mf.close()

print("✅ Fichier MIDI exporté sous le nom 'output.mid'")


✅ Fichier MIDI exporté sous le nom 'output.mid'


In [ ]:
partition

[[],
 [34],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [31],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]